In [809]:
import pandas as pd
import numpy as np
import pylab
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

In [810]:
########Begin Pre Processing the Happiness Scores CSV#########
# Read in the Happiness CSV files
happy_16 = pd.read_csv('2016.csv')
happy_18 = pd.read_csv('2018.csv')

# Print top of table as example
happy_16.head()

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Denmark,Western Europe,1,7.526,7.460,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,Switzerland,Western Europe,2,7.509,7.428,7.590,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,Iceland,Western Europe,3,7.501,7.333,7.669,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137
3,Norway,Western Europe,4,7.498,7.421,7.575,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465
4,Finland,Western Europe,5,7.413,7.351,7.475,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596


In [811]:
# Create new tables out of the relvant colums from the database
Happy16 =  happy_16[['Country', 'Region', 'Happiness Score']].copy() # Create new table with relevant features for 2016
Happy18 =  happy_18[['Country or region', 'Score']].copy() # Create new table with relevant features for 2018
Happy18.columns = ['Country', 'Happiness Score'] # Change column names to make them consistent
Happy18 = pd.merge(Happy18, Happy16[['Country', 'Region']], on=['Country'], how='left') # Merge the regional information from 16 to 18
Happy18 = Happy18[['Country', 'Region', 'Happiness Score']] # Reorder columns
Happy18.head()

,Country,Region,Happiness Score
0,Finland,Western Europe,7.632
1,Norway,Western Europe,7.594
2,Denmark,Western Europe,7.555
3,Iceland,Western Europe,7.495
4,Switzerland,Western Europe,7.487


In [812]:
# Find and clean missing values for Region
print(set(Happy18['Country']) - set(Happy16['Country'])) # Get Set of missing values
print(set(Happy16['Region'])) # Get set of possible regions

# Match different spellings of country name
Happy16['Country'].replace('North Cyprus', 'Cyprus', inplace=True)
Happy18['Country'].replace('Northern Cyprus', 'Cyprus', inplace=True)
Happy18['Country'].replace('Trinidad & Tobago', 'Trinidad and Tobago', inplace=True)

# Assign values based on most plausible region
Happy18.loc[Happy18['Country'] == 'Cyprus', ['Region']] = 'Middle East and Northern Africa'
Happy18.loc[Happy18['Country'] == 'Trinidad and Tobago', ['Region']] = 'Latin America and Caribbean'
Happy18.loc[Happy18['Country'] == 'Lesotho', ['Region']] = 'Sub-Saharan Africa'
Happy18.loc[Happy18['Country'] == 'Central African Republic', ['Region']] = 'Sub-Saharan Africa'
Happy18.loc[Happy18['Country'] == 'Mozambique', ['Region']] = 'Sub-Saharan Africa'

# Check columns for missing values
Happy18.isnull().sum(axis = 0)


{'Lesotho', 'Trinidad & Tobago', 'Mozambique', 'Northern Cyprus', 'Central African Republic'}
{'Southeastern Asia', 'Sub-Saharan Africa', 'Central and Eastern Europe', 'Western Europe', 'Australia and New Zealand', 'North America', 'Latin America and Caribbean', 'Middle East and Northern Africa', 'Eastern Asia', 'Southern Asia'}


Country            0
Region             0
Happiness Score    0
dtype: int64

In [813]:
# Export tables to csv files for easy reuse
Happy16.to_csv('HappinessScores16.csv')
Happy18.to_csv('HappinessScores18.csv')

In [814]:
########Begin Pre Processing the GDP pre Capita Scores CSV#########
# Read and format csv file
gdpCap = pd.read_csv('gdpCap.csv')
newGDP =  gdpCap[['Country Name', '2016', '2018']].copy() # Create copy of the table with the relevant columns
newGDP.columns = ['Country', 'gdpCap16', 'gdpCap18'] # Change column names to make them consistent
#gdpCap.head()

In [815]:
# Check country name discrepancies
set(Happy16['Country']) - set(newGDP['Country'])
set(Happy18['Country']) - set(newGDP['Country'])

# Map country names to Happiness Score Country names where possible
newGDP['Country'].replace('Congo, Rep.', 'Congo (Brazzaville)', inplace=True)
newGDP['Country'].replace('North Macedonia', 'Macedonia', inplace=True)
newGDP['Country'].replace('Yemen, Rep.', 'Yemen', inplace=True)
newGDP['Country'].replace('Slovak Republic', 'Slovakia', inplace=True)
newGDP['Country'].replace("Cote d'Ivoire", 'Ivory Coast', inplace=True)
newGDP['Country'].replace('Kyrgyz Republic', 'Kyrgyzstan', inplace=True)
newGDP['Country'].replace('Hong Kong SAR, China',  'Hong Kong', inplace=True)
newGDP['Country'].replace('Venezuela, RB', 'Russia', inplace=True)
newGDP['Country'].replace('Congo, Dem. Rep.', 'Congo (Kinshasa)', inplace=True)
newGDP['Country'].replace('Korea, Rep.', 'South Korea', inplace=True)
newGDP['Country'].replace('Syrian Arab Republic', 'Syria', inplace=True)
newGDP['Country'].replace('Lao PDR', 'Laos', inplace=True)
newGDP['Country'].replace('Egypt, Arab Rep.', 'Egypt', inplace=True)
newGDP['Country'].replace('Cyprus', 'North Cyprus', inplace=True)
newGDP['Country'].replace('Russian Federation', 'Russia', inplace=True)

# Check Country names have updated correctly
set(Happy16['Country']) - set(newGDP['Country'])
set(Happy18['Country']) - set(newGDP['Country'])

# Copy newGDP table to CSV
newGDP.to_csv('GDP.csv')

In [816]:
# Perform inner join merge between newGDP and Happiness table to drop any remaining outstanding values
gdpHappy16 = pd.merge(Happy16, newGDP[['Country', 'gdpCap16']], on=['Country'], how='inner')
gdpHappy18 = pd.merge(Happy18, newGDP[['Country', 'gdpCap18']], on=['Country'], how='inner')

# Check for nulls
gdpHappy16[gdpHappy16.isna().any(axis=1)]
gdpHappy18[gdpHappy18.isna().any(axis=1)]

# Drop null rows
gdpHappy16 = gdpHappy16.dropna()
gdpHappy18 = gdpHappy18.dropna()

# Check for nulls
gdpHappy16.isnull().sum(axis = 0)
gdpHappy18.isnull().sum(axis = 0)


Country            0
Region             0
Happiness Score    0
gdpCap18           0
dtype: int64

In [817]:
########Begin Pre Processing the EPI Scores CSV#########
epi16 = pd.read_csv('EPI 2016 Scores.csv')
epi18 = pd.read_csv('EPI 2018 Scores.csv')
#epi16.head()

In [818]:
# Identify null values in the Data to choose which columns to drop
epi16.isnull().sum(axis=0)
epi18.isnull().sum(axis=0)

# Check for 0 values in the dataset
epi16.isna().sum(axis=0)
epi18.isna().sum(axis=0)

# Average data between AIR.current and APE.current as EH - Air Quality encompasses both
epi18['AIRQ.current'] = epi18[['AIR.current', 'APE.current']].mean(axis=1)

# Extract useful Data to new table
Epi16 =  epi16[['Country', 'EH - Air Quality', 'EV - Biodiversity and Habitat', 'EH - Water and Sanitation', 'EV - Water Resources']].copy()
Epi18 =  epi18[['country', 'AIRQ.current', 'BDH.current', 'H2O.current', 'WRS.current']].copy()

# Rename table column headers for consistency
Epi16.columns = ['Country', 'Air Quality', 'Biodiversity', 'Water Sanitation', 'Water Resource']
Epi18.columns = ['Country', 'Air Quality', 'Biodiversity', 'Water Sanitation', 'Water Resource']

In [819]:
# Compare country sets to match up countries
print(set(gdpHappy16['Country']) - set(Epi16['Country']))
print(set(gdpHappy18['Country']) - set(Epi18['Country']))

# Map countries on the EPI16 reports
Epi16['Country'].replace('Congo', 'Congo (Brazzaville)', inplace=True)
Epi16['Country'].replace('Viet Nam', 'Vietnam', inplace=True)
Epi16['Country'].replace('United States of America', 'United States', inplace=True)
Epi16['Country'].replace("Cote d'Ivoire", 'Ivory Coast', inplace=True)
Epi16['Country'].replace('Dem. Rep. Congo', 'Congo (Kinshasa)', inplace=True)
Epi16['Country'].replace('Kyrgyz Republic', 'Kyrgyzstan', inplace=True)

# Map countries on the EPI18 reports
Epi18['Country'].replace('Republic of Congo', 'Congo (Brazzaville)', inplace=True)
Epi18['Country'].replace('Viet Nam', 'Vietnam', inplace=True)
Epi18['Country'].replace('United States of America', 'United States', inplace=True)
Epi18['Country'].replace('Dem. Rep. Congo', 'Congo (Kinshasa)', inplace=True)
Epi18['Country'].replace('Kyrgyz Republic', 'Kyrgyzstan', inplace=True)

# Perform inner merge to add EPI data to Happiness scores
gdpEpiHappy16 = pd.merge(gdpHappy16, Epi16, on=['Country'], how='inner')
gdpEpiHappy18 = pd.merge(gdpHappy18, Epi18, on=['Country'], how='inner')

{'Hong Kong', 'Puerto Rico', 'Vietnam', 'United States', 'Kyrgyzstan', 'Congo (Brazzaville)', 'Kosovo', 'Congo (Kinshasa)', 'Ivory Coast'}
{'Hong Kong', 'Yemen', 'Vietnam', 'United States', 'Congo (Brazzaville)', 'Somalia', 'Kosovo', 'Congo (Kinshasa)'}


In [820]:
# Convert to csv
gdpEpiHappy16.to_csv('test16.csv')
gdpEpiHappy18.to_csv('test18.csv')

In [821]:
#########Add CO2 data##########
# Measured in tons of CO2 per capita
# Data got from https://knoema.com/atlas/ranks/CO2-emissions-per-capita?action=export&gadget=tranking-container
# Extra values input manually from https://knoema.com/EDGARED2019/global-ghg-and-co2-emissions?location=1000500&indicator=1000070&type=1000140&utm_source=datafinder&utm_medium=excel&utm_campaign=sourcelink&frequency=A&lastUpdated=1585308536760#
co2 = pd.read_csv('CO2data.csv')
co2Data =  co2[['Location', '2016', '2018']].copy()
co2Data.head()

,Location,2016,2018
0,Afghanistan,0.29,0.29
1,Albania,1.60,1.64
2,Algeria,3.85,3.94
3,Angola,1.16,0.98
4,Antigua and Barbuda,6.26,6.23


In [822]:
# Check unmapped countries
print(set(gdpEpiHappy16['Country']) - set(co2Data['Location']))
print(set(gdpEpiHappy18['Country']) - set(co2Data['Location']))

# Map countries
co2Data['Location'].replace('Congo', 'Congo (Brazzaville)', inplace=True)
co2Data['Location'].replace('Democratic Republic of the Congo', 'Congo (Kinshasa)', inplace=True)
co2Data['Location'].replace('Myanmar/Burma', 'Myanmar', inplace=True)
co2Data['Location'].replace('North Macedonia', 'Macedonia', inplace=True)

# Change column names
co2Data.columns = ['Country', '2016', '2018']

# Merge CO2 Data
final16= pd.merge(gdpEpiHappy16, co2Data[['Country', '2016']], on=['Country'], how='inner')
final18= pd.merge(gdpEpiHappy18, co2Data[['Country', '2018']], on=['Country'], how='inner')


{'Congo (Kinshasa)', 'Congo (Brazzaville)', 'Myanmar'}
{'Congo (Kinshasa)', 'Congo (Brazzaville)', 'Myanmar'}


In [823]:
# Final table format
final16 = final16.reindex(columns=['Country', 'Region', 'gdpCap16', '2016', 'Air Quality', 'Biodiversity', 'Water Sanitation', 'Water Resource', 'Happiness Score'])
final18 = final18.reindex(columns=['Country', 'Region', 'gdpCap18', '2018', 'Air Quality', 'Biodiversity', 'Water Sanitation', 'Water Resource', 'Happiness Score'])
final16.columns = ['Country', 'Region', 'GDP per Capita', 'CO2 per Capita', 'Air Quality', 'Biodiversity', 'Water Sanitation', 'Water Resource', 'Happiness Score']
final18.columns = ['Country', 'Region', 'GDP per Capita', 'CO2 per Capita', 'Air Quality', 'Biodiversity', 'Water Sanitation', 'Water Resource', 'Happiness Score']

# Too many 0 values in the Water Resource column so needs to be deleted
final16 = final16.drop(['Water Resource'], axis=1)
final18 = final18.drop(['Water Resource'], axis=1)

# Check for nulls
print(final16.isna().sum(axis = 0))
print(final18.isna().sum(axis = 0))

# Drops any duplicates countries
final16.drop_duplicates(subset = 'Country', keep=False, inplace=True)
final18.drop_duplicates(subset = 'Country', keep=False, inplace=True)

# Replace 0 value
final18['Water Sanitation'].replace(0, 0.001, inplace=True)

# Get final table shape
final16.describe()
final18.describe()

# Create total table
finalTotal = final16.append(final18, ignore_index=True)

# Export to csv
final16.to_csv('final16.csv')
final18.to_csv('final18.csv')
finalTotal.to_csv('finalTotal.csv')


Country             0
Region              0
GDP per Capita      0
CO2 per Capita      0
Air Quality         0
Biodiversity        0
Water Sanitation    0
Happiness Score     0
dtype: int64
Country             0
Region              0
GDP per Capita      0
CO2 per Capita      0
Air Quality         0
Biodiversity        0
Water Sanitation    0
Happiness Score     0
dtype: int64


In [824]:

normal16 = final16.copy()
normal18 = final18.copy()
normalTotal = finalTotal.copy()

# Scale data using MinMaxScaler funcion
scaler = MinMaxScaler()
def MinMax(df):
    for col in df.columns:
        if df[f'{col}'].dtype != 'object':
            df[[f'{col}']] = scaler.fit_transform(df[[f'{col}']])

MinMax(normal16)
MinMax(normal18)
MinMax(normalTotal)

In [825]:
# Export Normalised data as CSV
normal16.to_csv('normal16.csv')
normal18.to_csv('normal18.csv')
normalTotal.to_csv('normalTotal.csv')

In [826]:
# Create QQplot for each column
def QQPlot(df):
    for col in df.columns:
        if df[f'{col}'].dtype != 'object':
            sm.qqplot(df[f'{col}'], line='s', label=f'{col}')
            pylab.legend()
            pylab.show()